## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [1]:
import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\aisvarya\AppData\Local\Temp\ipykernel_13724\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [18]:
driver.get("https://webapps1.chicago.gov/buildingrecords/")
driver.find_element(By.ID, 'rbnAgreement1').click()
driver.find_element(By.ID, 'submit').click()

## Searching

Search for **400 E 41ST ST**.

In [19]:

driver.find_element(By.ID, "fullAddress").send_keys("400 E 41ST ST")
driver.find_element(By.ID, 'submit').click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [39]:
!pip install jsonschema
!pip install html5lib


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
# permits_df = pd.read_html(driver.page_source)[0]
# permits_df

pd.read_html(driver.page_source, parse_dates=True)[0]

,PERMIT #,DATE ISSUED,DESCRIPTION OF WORK
0,100961712,NaN,2017 Fire Service Upgrade Mandate On Five (5) ...
1,100845718,NaN,ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
2,100778302,NaN,PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
3,100721255,NaN,PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
4,100693399,NaN,INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
5,100665436,NaN,PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...
6,100610771,NaN,PERMIT EXPIRES ON 10/28/2015 ERECTION STARTS: ...
7,100581991,NaN,TRACE AND REPAIR BROKEN UNDERGROUND FEED TO EX...
8,100479194,NaN,INTERNALLY LIT SIGN CABINET ON SOUTH ELEVATION
9,100385721,NaN,RPACE CONCRETE SLAB WITH NEW AT GROUNGD FLOOR ...


## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.